In [2]:
import json
import pandas as pd
import numpy as np

In [1]:
import os
cwd = os.getcwd()

In [4]:
business_path_ori = cwd + "\\data\\yelp_academic_dataset_business.json"

In [3]:
covid_path =  cwd +"\\data\\yelp_academic_dataset_covid_features.json"
business_path_new = cwd + "\\data\\yelp_academic_dataset_business.json"
user_path =  cwd +"\\data\\yelp_academic_dataset_user.json"
review_path = cwd + "\\data\\yelp_academic_dataset_review.json"

## COVID data

In [10]:
covid_data = []
with open(covid_path, "r", encoding="utf-8") as f:
    Lines = f.readlines()
    for l in Lines:
        covid_data.append(json.loads(l))
covid = pd.DataFrame.from_dict(covid_data)

In [11]:
def highlight_json(s):
    if type(s) == str:
        highlight = json.loads(s)
        identifier_list = ""
        for item in highlight:
            identifier_list += item['identifier'] + ", "
        identifier_list = identifier_list.rsplit(', ', 1)[0]
        return identifier_list
    else:
        return s

str_to_int = {'FALSE':0, 'TRUE':1}

column_names = covid.columns
for col in column_names:
    if col != "business_id": 
        if col in ["highlights", "Covid Banner", "Temporary Closed Until", "Virtual Services Offered"]:
            covid.loc[covid[col] == "FALSE", col] = np.nan
            if col == "highlights":
                covid.highlights = covid.highlights.apply(highlight_json)
        else:
            covid[col] = covid[col].map(str_to_int)        

In [ ]:
covid.to_csv("covid.csv", index = False)

## Business Data

In [5]:
business_data = []
with open(business_path_new, "r", encoding="utf-8") as f:
    Lines = f.readlines()
    for l in Lines:
        business_data.append(json.loads(l))
business = pd.DataFrame.from_dict(business_data)

In [6]:
business['is_rest'] = 0

In [7]:
with open("yelp_restaurant_cat.txt", "r") as f:
    lines = f.readlines()
rest_cat = [s.replace("\n","") for s in lines]

def intersection(lst1, lst2): 
    return list(set(lst1) & set(lst2)) 

for i in range(business.shape[0]):
    cat = business.categories.iloc[i]
    if type(cat) == str:
        cat_list = cat.split(", ")
        if len(intersection(rest_cat, cat_list)) > 0:
            business.is_rest.iloc[i] = 1

C:\Users\hefzh\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [8]:
restaurant = business[business.is_rest == 1]

def attributes_json(s):
    if s == None:
        return np.nan
    if "RestaurantsPriceRange2" not in s.keys():
        return np.nan
    else:
        return s['RestaurantsPriceRange2']

restaurant["Price_Range"] = restaurant.attributes.apply(attributes_json)
restaurant = restaurant.drop(columns = ["address", "attributes", "hours"])

<ipython-input-8-eac201975f27>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  restaurant["Price_Range"] = restaurant.attributes.apply(attributes_json)


In [ ]:
restaurant.groupby("state").agg({'business_id':'count'}).plot(kind='bar')

In [ ]:
restaurant.to_csv("restaurant_2021.csv", index = False)

In [2]:
covid = pd.read_csv("covid.csv")
restaurant = pd.read_csv("restaurant_2021.csv")

In [20]:
# COVID is not connected with restaurant 2021 so below is unnecessary
def flatten(_2d_list):
    flat_list = []
    # Iterate through the outer list
    for element in _2d_list:
        if type(element) is list:
            # If the element is of type list, iterate through the sublist
            for item in element:
                flat_list.append(item)
        else:
            flat_list.append(element)
    return flat_list

# Converting highlights into separate columns
restaurant_covid.highlights = restaurant_covid.highlights.replace(np.nan, "NA")
highlights_cat = restaurant_covid.highlights.unique()
l = []
for i in highlights_cat:
    if i is not np.nan:
        l.append(i.split(", "))
l_flat = flatten(l)
l_flat_unique = []
for i in set(l_flat):
    if "during_covid_19" in i:
        l_flat_unique.append(i)
for col in l_flat_unique:
    restaurant_covid[col] = 0
for i, h in enumerate(restaurant_covid.highlights):
    for col in l_flat_unique:
        restaurant_covid[col].iloc[i] = 1 if col in h else 0 

In [ ]:
restaurant_covid

In [21]:
restaurant_covid.highlights

0        NA
1        NA
2        NA
3        NA
4        NA
         ..
62716    NA
62717    NA
62718    NA
62719    NA
62720    NA
Name: highlights, Length: 62721, dtype: object

In [ ]:
restaurant_covid.columns

In [ ]:
restaurant_covid.to_csv("restaurant_covid_2021.csv", index = False)